In [28]:
from langchain_community.document_loaders import TextLoader

from langchain_ollama import ChatOllama
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate


from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_classic.chains.retrieval import create_retrieval_chain


In [29]:
#data ingest

In [ ]:
def ingest():
    loader = TextLoader("/content/speech.txt")
    pages = loader.load()   

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=0
    )
    chunks = text_splitter.split_documents(pages)

    print(f"Split {len(pages)} documents into {len(chunks)} chunks.")

    embedding = FastEmbedEmbeddings()

    chroma = Chroma.from_documents(
        documents=chunks,
        embedding=embedding,
        persist_directory="./chroma_database",
        collection_name="speech_collection"
    )


In [108]:
ingest()

Split 1 documents into 1 chunks.


In [109]:
ingest()

Split 1 documents into 1 chunks.


In [110]:
def rag_chain():
    model = ChatOllama(model="mistral")

    prompt = ChatPromptTemplate.from_template("""
You are a friendly assistant. Answer the question based ONLY on the context below.
If you don't know the answer, reply: "No context available for this question: {input}"

Question: {input}
Context: {context}
Answer:
""")

    embedding = FastEmbedEmbeddings()

    vector_store = Chroma(
        persist_directory="./chroma_database",
        embedding_function=embedding,
        collection_name="speech_collection"
    )

    retriever = vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 1}
    )

    text_chain = create_stuff_documents_chain(model, prompt)
    retrieval_chain = create_retrieval_chain(retriever, text_chain)
    return retrieval_chain


In [111]:
def ask(query: str):
    #
    chain = rag_chain()
    # invoke chain
    result = chain.invoke({"input": query})
    # print results
    print(result["answer"])
    for doc in result["context"]:
        print("Source: ", doc.metadata["source"])

## Ask Question to get relevant information from the document

In [113]:
ask("you must take a stand against what ? ")

 You must take a stand against the belief in the sanctity of the shastras.
Source:  /content/speech.txt


In [81]:
!ollama list

NAME              ID              SIZE      MODIFIED       
mistral:latest    6577803aa9a0    4.4 GB    12 minutes ago    


In [47]:
!ollama serve

Error: listen tcp 127.0.0.1:11434: bind: address already in use


In [ ]:
ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIP6JKgZGprPpKfS7YwQK9r8Fg21E9EKvcDb0BXLXWwmM

In [52]:
!ollama pull mistral